# Imports

In [1]:
import pickle as pkl
import time
import sys
import numpy as np

from bonsai.data_loaders import load_data
from bonsai.net import Net
from bonsai.trainers import *
from bonsai.helpers import *
from bonsai.ops import commons, Zero

%load_ext autoreload
%autoreload 2

In [2]:
mem_stats()

'0.00B'

In [3]:
gpu_space = 8.5
batch_size = 64
patterns = [['r','n','na'],['r','n','na'],['r','n','na'],['r',',n','na'],['r','n','na'],['r','n','na'],['n','na']]
hypers = {
    'dataset':'CIFAR10',
    'classes':10,
    'scale':5,
    'half':False,
    'batch_size':batch_size,
    'multiplier':1,
    'patterns':patterns,
    'num_patterns':{'init':1,'final':len(patterns)},
    'nodes':4,
    'lr_schedule':
        {'lr_max': .01,
         'T': 600},
    'drop_prob':.25,
}
schedule = {'cycle_len':4,
            'transition_after':2,
            'n_cycles':4}
sizes = {1:1,
         2:.95,
         3:.775,
         4:.6,
         5:.275,
         6:.475}
start_size=2

### test size

In [75]:
# gpu_space = 8
# batch_size = 2048
# patterns = [['n','ra']]
# hypers = {
#     'dataset':'CIFAR10',
#     'classes':10,
#     'scale':1,
#     'half':False,
#     'batch_size':batch_size,
#     'multiplier':1,
#     'patterns':patterns,
#     'num_patterns':{'init':1,'final':3},
#     'nodes':1,
#     'lr_schedule':
#         {'lr_max': .01,
#          'T': 600},
#     'drop_prob':.25,
# }
# schedule = {'cycle_len':2,
#             'transition_after':1,
#             'n_cycles':2}
# sizes = {1:1.,
#          2:1,
#          3:1,
#          4:1,
#          5:1}
# start_size = 2

# Determine Height/Size Ratios
Check how a test model scales under the search params to ensure we don't overfill GPU

In [10]:
#print(sp_size_test(hypers['num_patterns']['final']-1,e_c=.3,remove_prune=True,print_model=True,verbose=True,**hypers))
print(sp_size_test(hypers['num_patterns']['final']-1,e_c=.475,add_pattern=True,remove_prune=True,print_model=True,**hypers))

CalledProcessError: Command '['python3', '/home/campus.ncl.ac.uk/b6070424/Documents/Programming/Python/bonsai_net/bonsai/size_tester.py']' returned non-zero exit status 1.

In [9]:
sizes = {}
for n in range(hypers['num_patterns']['init'],hypers['num_patterns']['final']):
    sizes[n]=[]
    remove_prune = (n==hypers['num_patterns']['final']-1)
    bst=BST(.2,1.)
    while bst.answer is None:
        print("{}: {:.3f}\r".format(n,bst.pos),end="")
        size = sp_size_test(n,e_c=bst.pos,add_pattern=True,remove_prune=remove_prune,**hypers)
        query = not (not size[1] and (size[0])<gpu_space)
        bst.query(query)
    sizes[n]=max(bst.passes)

if any([v for (k,v) in sizes.items() if v==1]):
    start_size = [k for (k,v) in sizes.items() if v==1][-1]+1
else:
    start_size = hypers['num_patterns']['init']
print("Comp Ratios:",*["\n{}->{}: {:.3f}".format(k,k+1,v) for (k,v) in sizes.items()])

Comp Ratios: 
1->2: 1.000 
2->3: 0.950 
3->4: 0.775 
4->5: 0.600 
5->6: 0.275 
6->7: 0.475


# Search

## Model Setup

In [6]:
def jn_print(x,end="\n"):
    print(x,end=end)
    with open("logs/jn_out.log","a") as f:
        f.write(x+end)
          
# init model
data, dim = load_data(hypers['batch_size'], hypers['dataset'])
model = Net(dim=dim, 
            classes=hypers['classes'], 
            scale=hypers['scale'],
            patterns=hypers['patterns'], 
            num_patterns=start_size,
            nodes=hypers['nodes'],
            drop_prob=hypers['drop_prob'],
            lr_schedule=hypers['lr_schedule'])
model.data = data
#model.save_genotype()
size, overflow = size_test(model, data)
print(model)
print("Est Size: {}{:.2f}GiB {}".format(">" if overflow else "", size, "(overflow)" if overflow else "")) 
if overflow:
    del model
    clean('Search init')    

Init: 26.00MiB
0: 2.04GiB
1: 3.89GiB
2: 5.75GiB
Tower 2: 5.75GiB
3: 5.87GiB
4: 6.42GiB
5: 7.28GiB
GP: 7.28GiB
Classifier: 7.28GiB
========================== NETWORK ==========================
==================== Bakelite Ares Miles ====================
                     :     Dim      :   Params   :   Comp   
Initializer          :              :    160     :          
Cell 0  (Normal)     :   32 x 32    :   41,209   :  100.0%  
Cell 1  (Normal)     :   32 x 32    :   41,210   :  100.0%  
Cell 2  (Normal)     :   32 x 32    :   41,211   :  100.0%  
 ↳ Aux Tower         :              :  327,690   :          
Cell 3  (Reduction)  :   64 x 16    :  139,708   :  100.0%  
Cell 4  (Normal)     :   64 x 16    :  139,709   :  100.0%  
Cell 5  (Normal)     :   64 x 16    :  139,710   :  100.0%  
 ↳ Classifier        :              :  163,850   :          
Total                :              : 1,034,457  :  100.0%  

Est Size: 7.28GiB 


## Model Search

In [7]:
wipe_output()
search_start = time.time()
transition = schedule['cycle_len']*schedule['transition_after']
rate = {'edge':.5,'input':.5}

# search loop
for n in range(start_size,hypers['num_patterns']['final']):
    print("===",n,"===")
    finish = False
    comp_ratio = sizes.get(n,0)
    edge_prune = 1 if n<hypers['num_patterns']['final'] else 0
    aim = comp_ratio*.9 if comp_ratio>.35 else comp_ratio*.66
    jn_print("=== {} Patterns. Target Comp: {:.2f}, Aim: {:.2f}".format(n, comp_ratio,aim))

    for tries in range(1,10):
        if tries == 1:
            epochs = schedule['cycle_len']*schedule['n_cycles']
            comp_lambdas = TransitionDict(
                {model.lr_scheduler.t: None, 
                 model.lr_scheduler.t+transition: {'edge': rate['edge']*edge_prune*tries,
                                                   'input': rate['input']*tries}})
        else:
            epochs = transition
            comp_lambdas = TransitionDict(
                {model.lr_scheduler.t: {'edge': rate['edge']*edge_prune*tries,
                                        'input': rate['input']*tries}})

        full_train(model, epochs, comp_lambdas=comp_lambdas, comp_ratio=aim, prune_interval=schedule['cycle_len'])
        clean(verbose=False)
        hard_comp = model.genotype_compression()[1]
        if hard_comp and hard_comp > sizes[n]:
            jn_print("Try {}. Restarting pruning at pattern {}. Target comp: {:.2f}/{:.2f}, Actual: {:.3f}".format(tries,n,comp_ratio,aim,hard_comp))
        else:
            finish = True
            break

    if 1:#finish:
        #model.save_genotype()
        clean("Pre-add")
        if n == hypers['num_patterns']['final']-1:
            print("\tremoving pruners")
            s_c,h_c,i_c = model.genotype_compression()
            model.remove_pruners(remove_input=True,remove_edge=True)
        if n != hypers['num_patterns']['final']:
            print("Adding next pattern:",n+1)
            model.add_pattern()
    else: 
        print("No progress after 10 tries, aborting.")
        clean()
        break

clean("Search End")
#print(size_test(model,data))
print("Search Time:",show_time(time.time()-search_start))
print(model)

=== 2 ===
=== 2 Patterns. Target Comp: 0.95, Aim: 0.85
=== Training Bakelite Ares Miles ===
Starting at 2019-11-20 10:46:53.414004
11/20/2019 10:46 AM
Init: 50.00MiB
0: 2.04GiB
1: 3.90GiB
2: 5.75GiB
Tower 2: 5.75GiB
3: 5.87GiB
4: 6.42GiB
5: 7.28GiB
GP: 7.28GiB
Classifier: 7.28GiB


KeyboardInterrupt: 

# Train

In [ ]:
model.detail_print()

In [ ]:
full_train(model, epochs=model.lr_scheduler.remaining);
clean()

In [ ]:
model.prune

## Random Search

In [ ]:
e_c,i_c=.25,1

In [ ]:
data, dim =load_data(hypers['batch_size'], hypers['dataset'])
model = Net(dim=dim, 
            classes=hypers['classes'], 
            scale=hypers['scale'],
            num_patterns=hypers['num_patterns']['final'],
            patterns=hypers['patterns'],
            nodes=hypers['nodes'],
            random_ops={'e_c':e_c,'i_c':i_c}, 
            drop_prob=hypers['drop_prob'],
            lr_schedule=hypers['lr_schedule'],
            prune=False)
model.data = data
model.save_genotype()
print(model)
print(size_test(model,data))

In [ ]:
full_train(model,hypers['lr_schedule']['T'])